Tutaj ja namagam się przetworzyć dane tekstowe w kształt macierzy (c) Ivan

In [16]:
import numpy as np
import pandas as pd
import nltk
import spacy
from matplotlib import pyplot as plt
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import PCA
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import norm, uniform
from pywsd.utils import lemmatize_sentence

pd.set_option('display.max_columns', None)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ivansmaliakou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ivansmaliakou/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [17]:
df = pd.read_csv('../data/2_merged/merged_data.csv')

In [18]:
text_data = df.loc[:, df.dtypes == 'object']

In [19]:
df['NARRATIVE'].isna().sum()

43

In [20]:
df['NARRATIVE'] = df['NARRATIVE'].fillna('')

In [21]:
nlp = spacy.load('en_core_web_sm')
narrative_sentences = []
for raw_sent in tqdm(df['NARRATIVE']):
    doc = nlp(''.join([word.lower() for word in raw_sent]))
    lemmatized_tokens = [token.lemma_ for token in doc]
    lemmatized_text = ' '.join(lemmatized_tokens)
    narrative_sentences.append(lemmatized_text)

100%|██████████| 8134/8134 [03:47<00:00, 35.81it/s]


In [22]:
narrative_sentences

['the release occur near a normally dry drainage ditch that only flow during heavy rainy condition . it be identify by the land owner who notify the operator .   the repair have be make and clean up . the cause of the leak be believe to have be external corrosion at any above ground exposure .',
 "on december 30 , 2022 , a centurion pipeline , l.p . ( cpl ) employee discover a leak on a 2 '' union . the cpl employee arrive on - site at 08:35 am and confirm the leak be inside cpl 's midkiff station locate near midland , in midland county , texas . three ( 3.4 ) barrel of crude oil be release and recover during the process . cpl remediate the contaminated soil on site . no injury , fatality or fire be associate with this incident . cpl 's investigation determine that that cross threading occur when the nut be thread onto the union body . there be visible damage to the thread on the male end of the body indicate that cross threading occur due to misalignment between the nut and the body w

In [23]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
tfidf_wm = tfidfvectorizer.fit_transform(narrative_sentences)
tfidf_tokens = tfidfvectorizer.get_feature_names_out()
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)

In [24]:
df_tfidfvect.sum(axis=0).sort_values(ascending=False).head(20)

valve            393.468904
leak             317.642908
line             306.344810
gas              302.396936
pipeline         287.248559
release          278.986152
pump             253.508477
station          232.002137
tank             229.311965
pipe             200.652560
approximately    199.510223
cause            189.457526
report           189.345526
oil              188.972007
service          180.709613
repair           167.013101
product          160.522691
crude            158.419888
pressure         158.360736
seal             158.013205
dtype: float64

In [25]:
pca = PCA(svd_solver='randomized', n_components=500)
reduced_tfidf = pca.fit_transform(df_tfidfvect)

In [26]:
reduced_tfidf.shape

(8134, 500)

In [27]:
column_names = [f'narrative_tfidf-PC-{i}' for i in range(500)]
tfidf = pd.DataFrame(reduced_tfidf, columns=column_names)
joined_df = df.join(tfidf)

In [28]:
joined_df

,case_idx,data_source,case_lat,case_lon,case_date,INSTALLATION_YEAR,COMMODITY_RELEASED_TYPE,INTENTIONAL_RELEASE_BBLS,UNINTENTIONAL_RELEASE_BBLS,FATAL,INJURE,ON_OFF_SHORE,IGNITE_IND,EXPLODE_IND,NUM_PUB_EVACUATED,FEDERAL,LOCATION_TYPE,CROSSING,ITEM_INVOLVED,MATERIAL_INVOLVED,EST_COST_OPER_PAID,EST_COST_GAS_RELEASED,EST_COST_PROP_DAMAGE,EST_COST_EMERGENCY,EST_COST_ENVIRONMENTAL,EST_COST_OTHER,CAUSE,CAUSE_DETAILS,NARRATIVE,SYSTEM_PART_INVOLVED,INCIDENT_AREA_TYPE,PIPE_FACILITY_TYPE,inst_age_in_days,RELEASE_TYPE,COULD_BE_HCA,ACCIDENT_PSIG,MOP_PSIG,accident_pressure_as_%_mop_psig,PIPELINE_FUNCTION,SCADA_IN_PLACE_IND,INVESTIGATION_STATUS,EMPLOYEE_DRUG_TEST_IND,CONTRACTOR_DRUG_TEST_IND,zone,TAVG,narrative_tfidf-PC-0,narrative_tfidf-PC-1,narrative_tfidf-PC-2,narrative_tfidf-PC-3,narrative_tfidf-PC-4,narrative_tfidf-PC-5,narrative_tfidf-PC-6,narrative_tfidf-PC-7,narrative_tfidf-PC-8,narrative_tfidf-PC-9,narrative_tfidf-PC-10,narrative_tfidf-PC-11,narrative_tfidf-PC-12,narrative_tfidf-PC-13,narrative_tfidf-PC-14,narrative_tfidf-PC-15,narrative_tfidf-PC-16,narrative_tfidf-PC-17,narrative_tfidf-PC-18,narrative_tfidf-PC-19,narrative_tfidf-PC-20,narrative_tfidf-PC-21,narrative_tfidf-PC-22,narrative_tfidf-PC-23,narrative_tfidf-PC-24,narrative_tfidf-PC-25,narrative_tfidf-PC-26,narrative_tfidf-PC-27,narrative_tfidf-PC-28,narrative_tfidf-PC-29,narrative_tfidf-PC-30,narrative_tfidf-PC-31,narrative_tfidf-PC-32,narrative_tfidf-PC-33,narrative_tfidf-PC-34,narrative_tfidf-PC-35,narrative_tfidf-PC-36,narrative_tfidf-PC-37,narrative_tfidf-PC-38,narrative_tfidf-PC-39,narrative_tfidf-PC-40,narrative_tfidf-PC-41,narrative_tfidf-PC-42,narrative_tfidf-PC-43,narrative_tfidf-PC-44,narrative_tfidf-PC-45,narrative_tfidf-PC-46,narrative_tfidf-PC-47,narrative_tfidf-PC-48,narrative_tfidf-PC-49,narrative_tfidf-PC-50,narrative_tfidf-PC-51,narrative_tfidf-PC-52,narrative_tfidf-PC-53,narrative_tfidf-PC-54,narrative_tfidf-PC-55,narrative_tfidf-PC-56,narrative_tfidf-PC-57,narrative_tfidf-PC-58,narrative_tfidf-PC-59,narrative_tfidf-PC-60,narrative_tfidf-PC-61,narrative_tfidf-PC-62,narrative_tfidf-PC-63,narrative_tfidf-PC-64,narrative_tfidf-PC-65,narrative_tfidf-PC-66,narrative_tfidf-PC-67,narrative_tfidf-PC-68,narrative_tfidf-PC-69,narrative_tfidf-PC-70,narrative_tfidf-PC-71,narrative_tfidf-PC-72,narrative_tfidf-PC-73,narrative_tfidf-PC-74,narrative_tfidf-PC-75,narrative_tfidf-PC-76,narrative_tfidf-PC-77,narrative_tfidf-PC-78,narrative_tfidf-PC-79,narrative_tfidf-PC-80,narrative_tfidf-PC-81,narrative_tfidf-PC-82,narrative_tfidf-PC-83,narrative_tfidf-PC-84,narrative_tfidf-PC-85,narrative_tfidf-PC-86,narrative_tfidf-PC-87,narrative_tfidf-PC-88,narrative_tfidf-PC-89,narrative_tfidf-PC-90,narrative_tfidf-PC-91,narrative_tfidf-PC-92,narrative_tfidf-PC-93,narrative_tfidf-PC-94,narrative_tfidf-PC-95,narrative_tfidf-PC-96,narrative_tfidf-PC-97,narrative_tfidf-PC-98,narrative_tfidf-PC-99,narrative_tfidf-PC-100,narrative_tfidf-PC-101,narrative_tfidf-PC-102,narrative_tfidf-PC-103,narrative_tfidf-PC-104,narrative_tfidf-PC-105,narrative_tfidf-PC-106,narrative_tfidf-PC-107,narrative_tfidf-PC-108,narrative_tfidf-PC-109,narrative_tfidf-PC-110,narrative_tfidf-PC-111,narrative_tfidf-PC-112,narrative_tfidf-PC-113,narrative_tfidf-PC-114,narrative_tfidf-PC-115,narrative_tfidf-PC-116,narrative_tfidf-PC-117,narrative_tfidf-PC-118,narrative_tfidf-PC-119,narrative_tfidf-PC-120,narrative_tfidf-PC-121,narrative_tfidf-PC-122,narrative_tfidf-PC-123,narrative_tfidf-PC-124,narrative_tfidf-PC-125,narrative_tfidf-PC-126,narrative_tfidf-PC-127,narrative_tfidf-PC-128,narrative_tfidf-PC-129,narrative_tfidf-PC-130,narrative_tfidf-PC-131,narrative_tfidf-PC-132,narrative_tfidf-PC-133,narrative_tfidf-PC-134,narrative_tfidf-PC-135,narrative_tfidf-PC-136,narrative_tfidf-PC-137,narrative_tfidf-PC-138,narrative_tfidf-PC-139,narrative_tfidf-PC-140,narrative_tfidf-PC-141,narrative_tfidf-PC-142,narrative_tfidf-PC-143,narrative_tfidf-PC-144,narrative_tfidf-PC-145,narrative_tfidf-PC-146,narrative_tfidf-PC-147,narrative_

In [29]:
tfidf.to_csv('../tf-idf.ivan.csv')